In [4]:
# read ~/Downloads/pg.csv
import pandas as pd

df = pd.read_csv('~/Downloads/pg.csv')
len(set(df['essay_url']))

215

In [29]:
import os
import re
from IPython.display import Markdown, display

# Get a list of all files in the directory
files = os.listdir('./graham-essays/essays')


# Iterate over each file
for idx, file_name in enumerate(files):
    # Skip non .md files
    if not file_name.endswith('.md'):
        continue

    # Load the text
    with open(f'./graham-essays/essays/{file_name}', 'r') as file:
        text = file.read()

    # Extract notes
    notes = re.findall(r'<a name=(.*?)>\[(.*?)\]</a> (.*?)\n', text)

    # Reformat title
    title = re.search(r'# (.*?)\n', text).group(1)
    number, name = title.split(' ', 1)
    text = text.replace(f'# {title}', f'Essay #{number}: {name}')

    # Replace every occurrence of "rev." with "revised"
    text = text.replace('rev.', 'revised')

    # Remove image links, typically at the beginning as banners
    text = re.sub(r'\[\]\(.*?\)\s*', '', text)

    # Find all quoted paragraphs with many ">" and reformat them with quadruple quotes
    text = re.sub(r'( > .*\n)+', lambda match: '""""' + match.group(0).replace(' > ', ' ').replace('\n', ' ') + '""""\n', text)

    # Replace note references in the text with the parenthetical notes
    for note in notes:
        note_name, note_number, note_content = note
        text = text.replace(f'[{note_number}](#{note_name})', f'(SIDENOTE: {note_content.strip()})')

    # Remove the notes section
    notes_start = text.find('**Notes**')
    notes_end = text.find('**Thanks**')
    text = text[:notes_start] + text[notes_end:]

    # Remove translation links
    text = re.sub(r'\[.*?Translation.*?\]\(.*?\)\s*\n', '', text, flags=re.IGNORECASE)

    # Remove lines containing "* * *"
    text = re.sub(r'\* \* \*\n', '', text)

    # Save the updated text
    cleaned_file_name = file_name.replace('.md', '_cleaned.md')
    with open(f'./cleaned-essays/{cleaned_file_name}', 'w') as file:
        file.write(text)

    # pretty-print in jupyter notebook
    # display(Markdown(text))
    
    if idx == 10:
        break
